## Feature Extraction Notebook
---

This notebook is used to extract features from the data. The audio is divided into windows of 1 second. The features extracted from each audio window are:
- MFCC (Mel-Frequency Cepstral Coefficients): 20 coefficients
- Chroma STFT: 1 coefficient
- RMS (Root Mean Square) Energy: 1 coefficient
- Spectral Centroid: 1 coefficient
- Spectral Bandwidth: 1 coefficient
- Spectral Roll-off: 1 coefficient
- Zero Crossing Rate: 1 coefficient

In [1]:
# import all the functions
from utils import *
import numpy as np

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

# -------- tqdm DARK THEME --------

### Features Extraction

In [3]:
# set the paths to data
BASE_DIR = '../dataset/'
ARTIFACTS_DIR = BASE_DIR + 'artifacts/'
EXTRAHLS_DIR = BASE_DIR + 'extrahls/'
MURMURS_DIR = BASE_DIR + 'murmurs/'
NORMALS_DIR = BASE_DIR + 'normals/'
EXTRASTOLES_DIR = BASE_DIR + 'extrastoles/'

# paths to save the features
FEATURES_DIR = '../features/'

In [4]:
# FEATURES EXTRACTION
paths = [ARTIFACTS_DIR, EXTRAHLS_DIR, MURMURS_DIR, NORMALS_DIR, EXTRASTOLES_DIR]
names = ['artifacts', 'extrahls', 'murmurs', 'normals', 'extrastoles']
window_length = 1
sample_rate = 'mix'
n_mfcc = 20
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for i, path in enumerate(paths):
     print(i)
     
     # Save the features to a file
     name = f'{names[i]}_{window_length}_{sample_rate}.npz'
     
     if os.path.exists(FEATURES_DIR + name):
          print('The features have already been extracted')
     else:
          print(f'Extracting features from {path}')
          features = extract_features(path, label = i, frame_length = window_length, n_mfcc=n_mfcc)
          
          # Stack the features into a single tensor
          features = torch.cat(features, dim=0)
          print(f'The shape of the {path} features tensor is: {features.shape}')
          
          # if returns None continue
          if save_features(features, FEATURES_DIR, name) == None:
               continue
    
     print('Features extracted and saved')

          

0
Extracting features from ../dataset/artifacts/


Extraction in progress:   0%|          | 0/92 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...


/opt/homebrew/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...


...

No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs/ features tensor is: torch.Size([1149, 28])
Saving features...
3
Extracting features from ../dataset/normals/


Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__296_1311682952647_A1.wav. Skipping...
Converting stereo audio to mono for normal_2023_3.wav...
Converting stereo audio to mono for normal_2023_2.wav...
Converting stereo audio to mono for normal_2023_0.wav...
Converting stereo audio to mono for normal_2023_1.wav...
Finished processing all files.

The shape of the ../dataset/normals/ features tensor is: torch.Size([2161, 28])
Saving features...
4
Extracting features from ../dataset/extrastoles/


Extraction in progress:   0%|          | 0/46 [00:00<?, ?it/s]

Finished processing all files.

The shape of the ../dataset/extrastoles/ features tensor is: torch.Size([247, 28])
Saving features...


### Show the features statistics

In [5]:
artifacts_features = np.load(FEATURES_DIR + 'artifacts_1.npz')['X']
max = artifacts_features.max()
min = artifacts_features.min()
mean = artifacts_features.mean()
std = artifacts_features.std()
print('artifacts features')
print(f'Max: {max}, Min: {min}, Mean: {mean}, Std: {std}      -       Shape: {artifacts_features.shape}\n')

extrahls_features = np.load(FEATURES_DIR + 'extrahls_1.npz')['X']
max = extrahls_features.max()
min = extrahls_features.min()
mean = extrahls_features.mean()
std = extrahls_features.std()
print('extrahls features')
print(f'Max: {max}, Min: {min}, Mean: {mean}, Std: {std}       -       Shape: {extrahls_features.shape}\n')

murmur_features = np.load(FEATURES_DIR + 'murmurs_1.npz')['X']
max = murmur_features.max()
min = murmur_features.min()
mean = murmur_features.mean()
std = murmur_features.std()
print('murmur features')
print(f'Max: {max}, Min: {min}, Mean: {mean}, Std: {std}      -       Shape: {murmur_features.shape}\n')

normal_features = np.load(FEATURES_DIR + 'normals_1.npz')['X']
max = normal_features.max()
min = normal_features.min()
mean = normal_features.mean()
std = normal_features.std()
print('normal features')
print(f'Max: {max}, Min: {min}, Mean: {mean}, Std: {std}         -       Shape: {normal_features.shape}\n')

extrastoles_features = np.load(FEATURES_DIR + 'extrastoles_1.npz')['X']
max = extrastoles_features.max()
min = extrastoles_features.min()
mean = extrastoles_features.mean()
std = extrastoles_features.std()
print('extrastoles features')
print(f'Max: {max}, Min: {min}, Mean: {mean}, Std: {std}            -       Shape: {extrastoles_features.shape}')

FileNotFoundError: [Errno 2] No such file or directory: '../features/artifacts_1.npz'

In [ ]:
torch.ones(2,2)*0

tensor([[0., 0.],
        [0., 0.]])